# **1. BILBIOTHEQUES**

In [433]:
import json
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

---

# **2. EXTRACTION DES DONNEES BRUTES**

In [434]:
# Fonction pour extraire les données depuis le JSONL
def extract_data(file_path, max_rows=50):
    data = []
    
    with open(file_path, 'r') as f:
        for idx, line in enumerate(f):
            if idx >= max_rows:
                break

            position = json.loads(line)
            fen = position.get("fen")

            for eval_data in position.get("evals", []):
                knodes = eval_data.get("knodes")
                depth = eval_data.get("depth")
                
                for pv in eval_data.get("pvs", []):
                    cp = pv.get("cp")
                    mate = pv.get("mate")
                    line = pv.get("line")

                    data.append({
                        "fen": fen,
                        "knodes": knodes,
                        "depth": depth,
                        "cp": cp,
                        "mate": mate,
                        "line": line
                    })

    return pd.DataFrame(data)

In [435]:
# Utilisation de la fonction
file_path = 'data\lichess_db_eval.jsonl'  # Remplacez par le chemin de votre fichier
data_raw = extract_data(file_path, max_rows=500000)
data_raw

,fen,knodes,depth,cp,mate,line
0,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,200973,39,58.0,NaN,f7g7 e6e2 h8d8 e2d2 b7b5 c4e6 g7f6 e6b3 a6a5 a2a3
1,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,62.0,NaN,f7g7 e6e2 b7b5 c4b3 h8d8 e2d2 a6a5 a2a3 g7f6 d1e1
2,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,151.0,NaN,h8d8 d1e1 a6a5 a2a3 b7b5 c4a2 c6d7 e6e7 f7g6 e1f2
3,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,59730,31,64.0,NaN,f7g7 e6e2 g7g6 d1c2 h8d8 e2d2 g6f6 a2a3 b7b5 c4a2
4,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,59730,31,134.0,NaN,h8d8 d1e1 a6a5 a2a3 b7b5 c4b3 a5a4 b3a2 c6d7 e6h6
...,...,...,...,...,...,...
2330094,4r1k1/2R3b1/3pp1p1/1r3pBp/R6P/1P4P1/4PP2/6K1 w...,6342,20,54.0,NaN,e2e3 b5b3 a4a7 g7e5 c7h7 e8a8 a7d7 b3b2 h7e7 e5g3
2330095,4r1k1/2R3b1/3pp1p1/1r3pBp/R6P/1P4P1/4PP2/6K1 w...,6342,20,42.0,NaN,b3b4 e8b8 c7d7 b5b4 a4a7 g7d4 a7c7 d6d5 d7e7 b4b6
2330096,4r1k1/2R3b1/3pp1p1/1r3pBp/R6P/1P4P1/4PP2/6K1 w...,6342,20,35.0,NaN,a4a3 g7b2 a3a7 b2d4 a7a4 d4b6 c7d7 b5b3 a4a6 b6c5
2330097,4r1k1/2R3b1/3pp1p1/1r3pBp/R6P/1P4P1/4PP2/6K1 w...,6342,20,35.0,NaN,a4a7 g7d4 a7a4 d4b6 c7d7 b5b3 a4a6 b6c5 a6c6 b3c3


| Type            | Description                                                                                                        |
|-----------------|--------------------------------------------------------------------------------------------------------------------|
| **knodes**      | Nombre de milliers de nœuds analysés par stockfish pendant la recherche de la position.                   |
| **depth**       | Profondeur de la recherche, indiquant le nombre de coups (ou niveaux) explorés par le moteur.                     |
| **cp**          | Évaluation en centipions, représentant un avantage matériel en fonction du côté actif (positif pour Blancs, négatif pour Noirs). |
| **mate**        | Évaluation de mat. Si une valeur est donnée, elle indique le nombre de coups restants avant que le mat soit atteint. |

---

# **3. ML ENGEENERING**

## 3.1 Analyse des Variantes

Stockfish utilie différents parametre dans son algorythme, notamment le **nombre de noeuf (knodes)** et le la **profondeur (depth)**. En fonction de ces différents parametres, stockfish calcul le **centipions (cp)** et **l'évaluation du mate (mate)**.  

Après ces évaluations, stockfish peut alors parfois prédire des coups différents pour une même position, on appelle cela des **variantes**. Ces coups sont différents mais ont un impacte similaire ou quasi-similaire sur l'avantage qu'il donne a celui qui le joue (en terme d'evaluation de la position). 

Certains joueurs vont être plus a l'aise dans une variante plûtot qu'une autre en fonction de leurs styles de jeu ou de leurs connaissances, mais pour une IA cela n'as pas d'importance ! Toutefois, notre source de donnée nous propose plusieurs analyse de stockfish pour chaques position, apportant son lot de variante.

Nous pouvons alors utiliser 2 méthodes pour garder la meilleur variante :

### **Méthode 1 : Analyse des Parametres et de l'Evaluation** 
   1. **Profondeur de recherche (depth)** :
      - Plus la profondeur est grande, plus l'analyse est précise.
      - Priorisez les évaluations avec la profondeur maximale atteinte.

   2. **Centipions (cp) ou mat (mate)** :
      - Si une évaluation donne un **mate**, elle est prioritaire, car un mat forcé est absolu.
      - En absence de **mate**, choix d'évaluation avec la valeur **cp** la plus élevée pour le joueur actif

   3. **Nombre de nœuds analysés (knodes)** :
      - Si plusieurs évaluations ont la même profondeur, celle ayant exploré le plus grand nombre de nœuds est théoriquement plus fiable.

   4. **En cas d'égalité parfaite**:
      - En cas d’égalité sur les autres critères, utilisez une évaluation arbitraire.

### **Méthode 2 : Coup le Plus Populaire** 
blablablabla


Extrait le premier coup de la ligne (donc le prochain coup a jouer) pour en créer une variable a part

In [436]:
def extract_next_moove(df) :
    # Extraction du premier groupe de mots de la colonne 'line'
    df["analysed_best_move"] = df["line"].str.extract(r'(\S+)', expand=False)
    return(df)

Créer la variable du le coup le plus cité dans les prédictions d'une même position

In [437]:
def most_popular_predict_V1(df):
    most_popular_moves = (
        df.groupby('fen')['analysed_best_move']
        .agg(lambda x: x.value_counts().idxmax())  # Trouver le coup le plus fréquent
        .rename("most_popular_move_of_the_category")
    )

    # Ajouter cette information dans le DataFrame original
    df = df.merge(most_popular_moves, on='fen', how='left')
    return df

Ne garde que la ligne avec les critères d'évualiations les plus hauts et les parametres d'analyse de stockfish les plus "pousser"

In [438]:
def filter_best_predict(df):
    filtered_rows = []

    # Grouper par position FEN
    for fen, group in df.groupby('fen'):
        # Étape 1 : Sélectionner les évaluations avec la profondeur maximale
        max_depth = group['depth'].max()
        best_evals = group[group['depth'] == max_depth]
        
        # Étape 2 : Priorité à une évaluation de mat si elle existe
        if 'mate' in best_evals and best_evals['mate'].notna().any():
            best_row = best_evals.loc[best_evals['mate'].idxmin()]
        else:
            # Sinon, maximiser la valeur `cp`
            best_row = best_evals.loc[best_evals['cp'].idxmax()]
        
        # Ajouter la meilleure évaluation à la liste
        filtered_rows.append(best_row)
    
    # Créer un nouveau DataFrame avec les meilleures évaluations
    return pd.DataFrame(filtered_rows).reset_index(drop=True)

Supprime les colonnes inutiles

In [439]:
def drop_usuless_columns(df) :
    df = df.drop(columns=['line','mate','cp','depth','knodes'])
    return df

Application des fonctions

In [440]:
data_cleaned_V1 = extract_next_moove(data_raw)
data_cleaned_V1 = most_popular_predict_V1(data_cleaned_V1)
data_cleaned_V1 = filter_best_predict(data_cleaned_V1)
data_cleaned_V1 = drop_usuless_columns(data_cleaned_V1)
data_cleaned_V1

,fen,analysed_best_move,most_popular_move_of_the_category
0,1B1b4/8/2pP2p1/5p2/k7/8/5PK1/8 b - -,d8b6,d8b6
1,1B1k3r/p4ppp/5n2/2p1N3/1P6/8/P1P2PbP/2K1R3 b - -,g2d5,h8e8
2,1B1k4/2P2pp1/7p/1n6/8/7P/5PP1/5K2 b - -,d8e8,b5c7
3,1B1k4/2n2pp1/7p/8/8/7P/5PP1/5K2 w - -,b8c7,b8c7
4,1B1k4/5pp1/2P4p/1n6/8/7P/5PP1/5K2 w - -,h3h4,h3h4
...,...,...,...
499995,rrrrkrrr/ppp1p1pp/8/5p2/4P3/5P2/PPP3PP/RRRRKRR...,d8d6,d8d6
499996,rrrrkrrr/ppppp1pp/8/5p2/4P3/3P4/PPP2PPP/RRRRKR...,d7d5,d7d5
499997,rrrrkrrr/ppppp1pp/8/5p2/8/3P4/PPP1PPPP/RRRRKRR...,f2f4,f2f4
499998,rrrrkrrr/pppppppp/8/8/8/3P4/PPP1PPPP/RRRRKRRR ...,f7f5,f7f5


Export des Données Netoyées en CSV

In [441]:
data_cleaned_V1.to_csv('data/data_cleaned.csv')